In [3]:
import os
from tqdm import tqdm
import cv2
import numpy as np

Variables to change

In [85]:
name_query = 'qsd1_w5'
test_image = '19'

In [86]:
# Constant arguments
name_db = 'BBDD'
dir_base = '../../'
results_name = 'results'
aux_name = 'aux'


# Directories assignment (always end with /)
dir_db = f'{dir_base}{name_db}/' 
dir_query = f'{dir_base}{name_query}/'
dir_museum = f'{dir_base}museum/'
dir_results = f'{dir_query}{results_name}/'
dir_db_aux = f'{dir_db}{aux_name}/'
dir_query_aux = f'{dir_query}{aux_name}/'
dir_aux = f'{dir_base}{aux_name}/'
new_dirs = [dir_results, dir_db_aux, dir_query_aux, dir_aux]

for dir in new_dirs:
    try:
        os.makedirs(dir)
    except FileExistsError:
        # Directory already exists
        pass

In [6]:
def ssim(img1, img2) -> float:
    """
    It takes two images, calculates the mean of the squared difference between the two images, and then
    divides that by the mean of the squared difference between the two images
    
    :param img1: The first image being compared
    :param img2: The image to be compared to the original image
    :return: The mean of the ssim_map
    """
    C1 = (0.01 * 255)**2
    C2 = (0.03 * 255)**2

    img1 = img1.astype(np.float64)
    img2 = img2.astype(np.float64)
    kernel = cv2.getGaussianKernel(11, 1.5)
    window = np.outer(kernel, kernel.transpose())

    mu1 = cv2.filter2D(img1, -1, window)[5:-5, 5:-5]  # valid
    mu2 = cv2.filter2D(img2, -1, window)[5:-5, 5:-5]
    mu1_sq = mu1**2
    mu2_sq = mu2**2
    mu1_mu2 = mu1 * mu2
    sigma1_sq = cv2.filter2D(img1**2, -1, window)[5:-5, 5:-5] - mu1_sq
    sigma2_sq = cv2.filter2D(img2**2, -1, window)[5:-5, 5:-5] - mu2_sq
    sigma12 = cv2.filter2D(img1 * img2, -1, window)[5:-5, 5:-5] - mu1_mu2

    ssim_map = ((2 * mu1_mu2 + C1) * (2 * sigma12 + C2)) / ((mu1_sq + mu2_sq + C1) * (sigma1_sq + sigma2_sq + C2))
    
    return ssim_map.mean()

def calculate_ssim(img1, img2) -> float:
    '''calculate SSIM
    the same outputs as MATLAB's
    img1, img2: [0, 255]
    '''
    if not img1.shape == img2.shape:
        raise ValueError('Input images must have the same dimensions.')
    if img1.ndim == 2:
        return ssim(img1, img2)
    elif img1.ndim == 3:
        if img1.shape[2] == 3:
            ssims = []
            for i in range(3):
                ssims.append(ssim(img1, img2))
            return np.array(ssims).mean()
        elif img1.shape[2] == 1:
            return ssim(np.squeeze(img1), np.squeeze(img2))
    else:
        raise ValueError('Wrong input image dimensions.')

def noise_ckeck_removal(image, f_name):
    """
    It takes an image and a file name as input, and returns the same image if the similarity between the
    original image and the denoised image is greater than 0.65, otherwise it returns the denoised image
    
    :param image: The image to be denoised
    :param f_name: The name of the image file
    :return: The image is being returned.
    """
    image_denoised = cv2.medianBlur(image, 3)
    ssim = calculate_ssim(image, image_denoised)
    if(ssim < 0.65):
        cv2.imwrite(dir_query + dir_query_aux + f_name + '_denoised.png', image_denoised)
        return image_denoised
    return image

In [7]:
def get_avg_corners_color(image):
    # Get average color of the image corners
    # top left
    avg_color_per_row = np.average(image[0:10, 0:10], axis=0)
    avg_color = np.average(avg_color_per_row, axis=0)
    avg_color_tl = avg_color
    # top right
    avg_color_per_row = np.average(image[0:10, image.shape[1]-10:image.shape[1]], axis=0)
    avg_color = np.average(avg_color_per_row, axis=0)
    avg_color_tr = avg_color
    # bottom left
    avg_color_per_row = np.average(image[image.shape[0]-10:image.shape[0], 0:10], axis=0)
    avg_color = np.average(avg_color_per_row, axis=0)
    avg_color_bl = avg_color
    # bottom right
    avg_color_per_row = np.average(image[image.shape[0]-10:image.shape[0], image.shape[1]-10:image.shape[1]], axis=0)
    avg_color = np.average(avg_color_per_row, axis=0)
    avg_color_br = avg_color
    # Get the average color of the corners
    avg_color_corners = (avg_color_tl + avg_color_tr + avg_color_bl + avg_color_br) / 4
    return avg_color_corners

In [8]:
def rotation_check(image, f_name):
    # Applying hough to detect lines
    image_cpy = image.copy()

    # Keep only clear edges of the image
    edges = cv2.Canny(image, 225, 250)

    # edges = cv2.Canny(image, 50, 150, apertureSize = 3)
    lines = cv2.HoughLinesP(edges, 1, np.pi / 180, 200)

    # See result of canny
    cv2.imwrite(dir_aux + f_name + '_canny.png', edges)

    # Get length of the lines
    # lengths = []
    angles = []

    # If there are no lines detected, return the original image
    if lines is None:
        return image, False, 0

    for line in lines:
        rho, theta = line[0]
        # Allow only from 315 to 45 degrees and 135 to 225 degrees (but we considere horizontal is pi / 2)
        if (theta > 0.25 * np.pi and theta < 0.75 * np.pi) or (theta > 1.25 * np.pi and theta < 1.75 * np.pi):
            a = np.cos(theta)
            b = np.sin(theta)
            x0 = a * rho
            y0 = b * rho
            pt1 = (int(x0 + 1000 * (-b)), int(y0 + 1000 * (a)))
            pt2 = (int(x0 - 1000 * (-b)), int(y0 - 1000 * (a)))
            # lengths.append(np.sqrt((pt1[0] - pt2[0]) ** 2 + (pt1[1] - pt2[1]) ** 2))
            angles.append(theta)
            cv2.line(image_cpy, pt1, pt2, (0, 0, 255), 2)
    

    cv2.imwrite(dir_aux + f_name + '_lines.png', image_cpy)
    # Get the longest line and its angle

    # print(lengths)
    print(angles)
    # Check if angles is empty
    if not angles:
        return image, False, 0
    
    # Get the most repeated angle
    angle = max(set(angles), key = angles.count)
    # Substract pi / 2 to get the angle of the line
    angle = angle - np.pi / 2
    # Get the angle in degrees
    angle_deg = angle * 180 / np.pi
    print(angle_deg)
    # Get the rotation matrix
    M = cv2.getRotationMatrix2D((image.shape[1] / 2, image.shape[0] / 2), angle_deg, 1)
    # Rotate the image
    rotated_image = cv2.warpAffine(image, M, (image.shape[1], image.shape[0]), borderValue=get_avg_corners_color(image))
    # Save the rotated image
    cv2.imwrite(dir_aux + f_name + '_rotated.png', rotated_image)
    # Print the angle in degrees in a file
    # with open(dir_aux + f_name + '_angle.txt', 'w') as f:
    #     f.write(str(angle_deg))
    return rotated_image, M, angle

In [104]:
def auto_canny(image, sigma=0.33):
	# compute the median of the single channel pixel intensities
	v = np.median(image)
	# apply automatic Canny edge detection using the computed median
	lower = int(max(0, (1.0 - sigma) * v))
	upper = int(min(255, (1.0 + sigma) * v))
	edged = cv2.Canny(image, lower, upper)
	# return the edged image
	return edged

def rotation_check_prob(image, f_name):
    # Applying hough to detect lines
    image_cpy = image.copy()
    height, width = image.shape[:2]

    # edges = auto_canny(image)
    # Apply a tight cannny to detect edges
    edges = cv2.Canny(image, 200, 250)

    lines = cv2.HoughLinesP(edges, rho = 1, theta = 1*np.pi/180, threshold = 150, minLineLength = 100, maxLineGap = width)

    # Save result of canny
    cv2.imwrite(dir_aux + f_name + '_canny.png', edges)

    # If there are no lines detected, return the original image
    if lines is None:
        return image, False, 0, None
    
    # Get length of the lines
    final_length, final_angle, final_line = None, None, None
    min_height = height
    for line in lines:
        x1, y1, x2, y2 = line[0]
        if  (y1 < height / 3 or y2 < height / 3) or (y1 > 2 * height / 3 or y2 > 2 * height / 3):
            # Get angle on radians
            angle = np.arctan2(y2 - y1, x2 - x1)
            length = np.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)
            mean_height = (y1 + y2) / 2
            
            # Plot lines in different colors for each part (range from -pi to pi)
            if (angle <= np.pi / 4 and angle >= - np.pi / 4) or (angle >= 3 * np.pi / 4 or angle <= - 3 * np.pi / 4):
                color = (0, 255, 0)
                if final_angle == None or mean_height < min_height:# length > final_length:
                    final_angle = angle
                    final_length = length
                    min_height = mean_height
                    final_line = [(x1, y1), (x2, y2)]
            else:
                color = (0, 0, 255)
            cv2.line(image_cpy, (x1, y1), (x2, y2), color, 2)

    if final_line:
        cv2.line(image_cpy, final_line[0], final_line[1], (255, 0, 0), 5)

    cv2.imwrite(dir_aux + f_name + '_lines.png', image_cpy)

    # Check if angles is empty
    if not final_angle:
        return image, False, 0, None
    
    # Get the angle in degrees considering range pi to -pi
    angle_deg = final_angle * 180 / np.pi

    print(angle_deg)
    # Get the rotation matrix
    M = cv2.getRotationMatrix2D((image.shape[1] / 2, image.shape[0] / 2), angle_deg, 1)
    # Rotate the image

    # Transform the image to BGRA to avoid black borders
    image = cv2.cvtColor(image, cv2.COLOR_BGR2BGRA)
    # rotated_image = cv2.warpAffine(image, M, (image.shape[1], image.shape[0]), borderValue=get_avg_corners_color(image))
    rotated_image = cv2.warpAffine(image, M, (image.shape[1], image.shape[0]), borderValue=cv2.BORDER_TRANSPARENT)
    # Save the rotated image
    cv2.imwrite(dir_aux + f_name + '_rotated.png', rotated_image)
    # Create a mask for the transparent pixels
    rotatation_mask = rotated_image[:,:,3] == 0
    # Format the mask to be binary
    rotatation_mask = rotatation_mask.astype(np.uint8) * 255
    cv2.imwrite(dir_aux + f_name + '_rotated_mask.png', rotatation_mask)
    
    # Transform the image back to BGR
    rotated_image = cv2.cvtColor(rotated_image, cv2.COLOR_BGRA2BGR)

    return rotated_image, M, final_angle, rotatation_mask

For 1 image

In [102]:
name_query = 'qsd1_w5'
test_image = '28'

In [103]:
f = f'{dir_query}000{test_image}.jpg'
f_name = f.split('/')[-1].split('.')[0]
image = cv2.imread(f)

image = noise_ckeck_removal(image,f_name)
cv2.imwrite(dir_aux + f_name + '_pre_rotation.png', image)
image, rotation_matrix, angle, rotatation_mask = rotation_check_prob(image, f_name)

For all the query

In [105]:
for filename in tqdm(os.scandir(dir_query)):
    f = filename.path
    f_name = f.split('/')[-1].split('.')[0]
    if f.endswith('.jpg'): 
        image = cv2.imread(f)

        image = noise_ckeck_removal(image,f_name)
        cv2.imwrite(dir_aux + f_name + '_pre_rotation.png', image)
        image, rotation_matrix, angle, rotatation_mask = rotation_check_prob(image, f_name)


4it [00:10,  2.08s/it]

-1.8830186182090713
-0.9979253257527998


10it [00:17,  1.25s/it]

-45.0


16it [00:18,  1.55it/s]

...

-1.0274771945295478


40it [00:56,  1.22s/it]

1.032244691565731


48it [01:05,  1.37s/it]


KeyboardInterrupt: 